In [1]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [2]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Load and clean the data
df = pd.read_csv('/workspaces/codespaces-jupyter/data/FTE Enrollment by Grade Fiscal Year2019-3 Data Report.csv', skiprows=5, encoding='latin1')
df.columns = df.columns.str.strip()

# Melt the dataframe for easier plotting
df_long = df.melt(id_vars=['School Name'], 
                  value_vars=[col for col in df.columns if 'Grade' in col], 
                  var_name='Grade', 
                  value_name='Enrollment')

# Start Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('📊 Georgia K-12 Enrollment Dashboard', style={'textAlign': 'center'}),

    html.Div([
        html.Label('Select a School:'),
        dcc.Dropdown(
            options=[{'label': school, 'value': school} for school in df['School Name'].unique()],
            id='school-dropdown',
            value=df['School Name'].iloc[0],
            style={'width': '80%'}
        )
    ], style={'padding': '20px'}),

    dcc.Graph(id='school-grade-bar'),

    html.Hr(),

    html.Div([
        html.Label('Select a Grade:'),
        dcc.Dropdown(
            options=[{'label': g, 'value': g} for g in df_long['Grade'].unique()],
            id='grade-dropdown',
            value='Grade 01',
            style={'width': '80%'}
        )
    ], style={'padding': '20px'}),

    dcc.Graph(id='grade-across-schools')
])

@app.callback(
    Output('school-grade-bar', 'figure'),
    Input('school-dropdown', 'value')
)
def update_school_chart(selected_school):
    school_data = df_long[df_long['School Name'] == selected_school]
    fig = px.bar(school_data, x='Grade', y='Enrollment', title=f'Enrollment by Grade: {selected_school}')
    return fig

@app.callback(
    Output('grade-across-schools', 'figure'),
    Input('grade-dropdown', 'value')
)
def update_grade_chart(selected_grade):
    grade_data = df[['School Name', selected_grade]].sort_values(selected_grade, ascending=False)
    fig = px.bar(grade_data, x=selected_grade, y='School Name', orientation='h', 
                 title=f'{selected_grade} Enrollment Across Schools')
    return fig

if __name__ == '__main__':
    app.run(debug=True)

